In [ ]:
import pandas as pd
import numpy as np
import requests

import selenium
print(selenium.__version__)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager 

from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as bs

import re
from datetime import date
import json

import time
import random
import os

x_username = '...Your X Username...'
x_password = '..Your X Password...'
x_email = '...Your X Email...'

def handle_security_check(bot):
    """보안 확인 화면에서 이메일을 입력하고 다음 단계로 진행"""
    try:
        # 이메일 입력 필드 찾기
        email_input = WebDriverWait(bot, 10).until(
            EC.presence_of_element_located((By.XPATH, '//input[@name="text"]'))
        )
        email_input.send_keys(x_email)  # 이메일 입력
        email_input.send_keys(Keys.RETURN)  # Enter 키 입력
        print("[Info] - 이메일 입력 완료")

        time.sleep(5)  # 다음 페이지 로딩 대기

    except Exception as e:
        print(f"보안 확인 X")


def login_X(bot, username, password):
    bot.get('https://x.com/i/flow/login')
    time.sleep(5)
    print("[Info] - Logging in...")
    username_input = WebDriverWait(bot, 10).until(
        EC.presence_of_element_located((By.XPATH, '//input[@name="text"]'))
    )
    username_input.send_keys(username)
    username_input.send_keys(Keys.RETURN)
    print('username_input done')
    time.sleep(5)
    
    handle_security_check(bot)
    
    password_input = WebDriverWait(bot, 10).until(
        EC.presence_of_element_located((By.XPATH, '//input[@name="password"]'))
    )
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN)
    print('password_input done')

    time.sleep(10)

4.34.2


In [ ]:
from datetime import datetime, timedelta

date_dict = {
    '2022': {'08-04', '08-05', '08-06', '08-07', '08-08', '08-09', '08-10'},
    '2023': {'08-03', '08-04', '08-05', '08-06', '08-07', '08-08', '08-09'},
    '2024': {'08-01', '08-02', '08-03', '08-04', '08-05', '08-06', '08-07'},
    '2025': {'07-31', '08-01', '08-02', '08-03', '08-04', '08-05', '08-06'},
    '2023': {'08-03', '08-04', '08-05', '08-06', '08-07', '08-08', '08-09'}
}

# datetime.date 객체로 변환
date_dict_dt = {
    year: {datetime.strptime(f"{year}-{md}", "%Y-%m-%d").date() for md in md_set}
    for year, md_set in date_dict.items()
}

# 변환된 dict 확인
for year, dates in date_dict_dt.items():
    for d in sorted(dates):
        print(f"since:{d}, until:{d + timedelta(days=1)}")


since:2025-07-31, until:2025-08-01
since:2025-08-01, until:2025-08-02
since:2025-08-02, until:2025-08-03
since:2025-08-03, until:2025-08-04
since:2025-08-04, until:2025-08-05
since:2025-08-05, until:2025-08-06
since:2025-08-06, until:2025-08-07


In [ ]:
service = Service()
options = webdriver.ChromeOptions()
options.add_argument("--lang=en")
# options.add_argument("--headless")
options.add_argument("--remote-debugging-port=9222")
options.add_argument("--window-size=1920,1080")  # 창 크기 지정
options.add_argument("--disable-gpu")  # GPU 비활성화 (Headless 안정성 증가)
options.add_argument("--disable-dev-shm-usage")  # 메모리 부족 문제 해결
options.add_argument('--no-sandbox')
options.add_argument("--log-level=3")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
# mobile_emulation = 
    # "userAgent": "Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexus 5 Build/JOP40D) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/90.0.1025.166 Mobile Safari/535.19"}
# options.add_experimental_option("mobileEmulation", mobile_emulation)

def scrape_tweets_by_date(d, bot):
    query = f"펜타 since:{d} until:{d + timedelta(days=1)}"
    url = f"https://x.com/search?q={query}&src=typed_query&f=live"
    bot.get(url)

    # 페이지 스크롤 반복 (트윗 더 불러오기)
    last_height = bot.execute_script("return document.body.scrollHeight")
    same_count = 0

    tweets_data = []

    for i in range(300):  # 시도 횟수 제한
        bot.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(np.random.uniform(np.random.uniform(5,10), np.random.uniform(40,60)))

        new_height = bot.execute_script("return document.body.scrollHeight")
        new_cards = bot.find_elements(By.XPATH, '//article[@data-testid="tweet"]')

        print(f"[{i+1}] 트윗 수: {len(new_cards)}")
        
        if new_height == last_height:
            same_count += 1
            if same_count >= 3:
                print("더 이상 새 트윗이 로드되지 않음. 중단.")
                break
        else:
            same_count = 0
            last_height = new_height
        
        for card in new_cards:
            try:
                text_elem = card.find_element(By.XPATH, './/div[@data-testid="tweetText"]')
                time_elem = card.find_element(By.XPATH, './/time')
                text = text_elem.text
                timestamp = time_elem.get_attribute("datetime")  # ISO 8601 포맷
                tweets_data.append({'text': text, 'datetime': timestamp})
            except:
                continue
    return tweets_data

def scrape_with_retries(date, bot, retries=3, wait_sec=1200):
    """트윗 스크랩을 여러 번 시도, 성공 시 데이터 반환"""
    for attempt in range(retries):
        tweets_data = scrape_tweets_by_date(date, bot)
        if len(tweets_data) >= 100:
            return tweets_data
        print(f"fail to scrape (attempt {attempt+1}), waiting {wait_sec} sec...")
        bot.quit()
        time.sleep(wait_sec)
        try:
            bot = webdriver.Chrome(service=service, options=options)
            bot.set_page_load_timeout(30) # Set the page load timeout to 15 seconds
            login_X(bot, username=x_username, password = x_password)
            time.sleep(5)
        except Exception as e:
            time.sleep(30)
            bot = webdriver.Chrome(service=service, options=options)
            bot.set_page_load_timeout(30) # Set the page load timeout to 15 seconds
            login_X(bot, username=x_username, password = x_password)
            time.sleep(5)
        

    return tweets_data  # 마지막 결과 반환


In [ ]:
for year, dates in date_dict_dt.items():  # .items()로 수정
    for d in sorted(dates):
        bot = webdriver.Chrome(service=service, options=options)
        bot.set_page_load_timeout(30) # Set the page load timeout to 15 seconds
        login_X(bot, username=x_username, password = x_password)
        time.sleep(5)
        print(d)
        tweets_data = scrape_with_retries(d, bot)
        time.sleep(600)
        bot.quit()
        pd.DataFrame(tweets_data).to_csv(f'./data/{d}.csv', index=False)

[Info] - Logging in...
username_input done
보안 확인 X
password_input done
2025-07-31
[1] 트윗 수: 12
[2] 트윗 수: 21
[3] 트윗 수: 24
[4] 트윗 수: 23
[5] 트윗 수: 18
[6] 트윗 수: 26
[7] 트윗 수: 25
[8] 트윗 수: 23
[9] 트윗 수: 24
[10] 트윗 수: 19
[11] 트윗 수: 16
[12] 트윗 수: 15
[13] 트윗 수: 18
[14] 트윗 수: 21
[15] 트윗 수: 21
[16] 트윗 수: 15
[17] 트윗 수: 15
[18] 트윗 수: 11
[19] 트윗 수: 18
[20] 트윗 수: 19
[21] 트윗 수: 17
[22] 트윗 수: 15
[23] 트윗 수: 20
[24] 트윗 수: 26
[25] 트윗 수: 21
[26] 트윗 수: 26
[27] 트윗 수: 24
[28] 트윗 수: 27
[29] 트윗 수: 19
[30] 트윗 수: 15
[31] 트윗 수: 21
[32] 트윗 수: 25
[33] 트윗 수: 30
[34] 트윗 수: 26
[35] 트윗 수: 16
[36] 트윗 수: 27
[37] 트윗 수: 24
[38] 트윗 수: 21
[39] 트윗 수: 13
[40] 트윗 수: 13
[41] 트윗 수: 13
더 이상 새 트윗이 로드되지 않음. 중단.
[Info] - Logging in...
username_input done
보안 확인 X
password_input done
2025-08-01
[1] 트윗 수: 14
[2] 트윗 수: 25
[3] 트윗 수: 24
[4] 트윗 수: 19
[5] 트윗 수: 23
[6] 트윗 수: 21
[7] 트윗 수: 20
[8] 트윗 수: 28
[9] 트윗 수: 21
[10] 트윗 수: 24
[11] 트윗 수: 23
[12] 트윗 수: 18
[13] 트윗 수: 18
[14] 트윗 수: 23
[15] 트윗 수: 26
[16] 트윗 수: 13
[17] 트윗 수: 19
[18] 트윗 수: 24
[19]

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff6ddac6b25+79621]
	GetHandleVerifier [0x0x7ff6ddac6b80+79712]
	(No symbol) [0x0x7ff6dd85c0ea]
	(No symbol) [0x0x7ff6dd8b2f56]
	(No symbol) [0x0x7ff6dd8b320c]
	(No symbol) [0x0x7ff6dd9065b7]
	(No symbol) [0x0x7ff6dd8db17f]
	(No symbol) [0x0x7ff6dd9033d0]
	(No symbol) [0x0x7ff6dd8daf13]
	(No symbol) [0x0x7ff6dd8a4151]
	(No symbol) [0x0x7ff6dd8a4ee3]
	GetHandleVerifier [0x0x7ff6ddd8683d+2962461]
	GetHandleVerifier [0x0x7ff6ddd80b5d+2938685]
	GetHandleVerifier [0x0x7ff6ddd9f71d+3064573]
	GetHandleVerifier [0x0x7ff6ddae0c6e+186446]
	GetHandleVerifier [0x0x7ff6ddae8a3f+218655]
	GetHandleVerifier [0x0x7ff6ddacf914+115956]
	GetHandleVerifier [0x0x7ff6ddacfac9+116393]
	GetHandleVerifier [0x0x7ff6ddab5ef8+10968]
	BaseThreadInitThunk [0x0x7fffdfdce8d7+23]
	RtlUserThreadStart [0x0x7fffe0c5c34c+44]
